In [1]:
import numpy as np
import json
import os
import pandas as pd
import matplotlib.pyplot as plt

from utils import load_env_file, set_mpl_configs
from utils import leave_percentile, distribution_analysis

load_env_file()
set_mpl_configs()

DATA_DIR = os.getenv('DATA_DIR')
print('DATA_DIR: {}'.format(DATA_DIR))

load env file
  root dir:
    /home/k/Repo/IBD-EDA
  current system:
    Linux
  load .env.linux
  loaded data dir:
    /home/k/Nutstore Files/毕设-EHR/DB
done.
set matplotlib configs
  font family:
    ['Times New Roman']
done.
DATA_DIR: /home/k/Nutstore Files/毕设-EHR/DB


In [2]:
with open('../data/ibd_demo.json', 'r') as f:
    data = json.loads(f.read())
    
both_ibd_patients: list = data['both_ibd']
only_uc_patiens: list = data['only_uc']
only_cd_patients: list = data['only_cd']


df = pd.read_csv(os.path.join(DATA_DIR, 'complication', 'Complications_Patients.csv'))
df_demography = df.groupby('subject_id').agg({
    'gender': 'first',
    'anchor_age': 'first',
    'anchor_year_group': 'first',
    'dod': 'first',
})

# df_demography = df_demography.loc[only_cd_patients, :]

In [3]:
df_demography.head()

,gender,anchor_age,anchor_year_group,dod
subject_id,,,,
10001186,F,46,2011 - 2013,None
10007174,M,70,2011 - 2013,None
10018852,M,19,2011 - 2013,None
10024331,M,72,2008 - 2010,2145-01-23
10025647,M,83,2008 - 2010,2181-06-16


In [4]:
def get_suspect_icd_list(threshold=300) -> list:
    stats = df.groupby(['subject_id', 'icd_code']).agg({
        'icd_code': ['count'],
    })
    stats.columns = ['count']
    stats = stats.reset_index()
    stats = stats.groupby('icd_code').agg({
        'subject_id': ['nunique']
    })
    stats.columns = ['count']
    return [_ for _ in stats[stats['count'] > threshold].index.tolist() if _ not in ['5550',
                                                                                     '5551',
                                                                                     '5552',
                                                                                     '5559',
                                                                                     '5560',
                                                                                     '5561',
                                                                                     '5562',
                                                                                     '5563',
                                                                                     '5564',
                                                                                     '5565',
                                                                                     '5566',
                                                                                     '5568',
                                                                                     '5569']]

In [5]:
2417 * 0.05

120.85000000000001

In [7]:
get_suspect_icd_list(120)

['00845',
 '0389',
 '2449',
 '25000',
 '2639',
 '2720',
 '2724',
 '2761',
 '2762',
 '27651',
 '27652',
 '2767',
 '2768',
 '27800',
 '2800',
 '2809',
 '2851',
 '28529',
 '2859',
 '2875',
 '28860',
 '30000',
 '3004',
 '3051',
 '311',
 '32723',
 '33829',
 '34690',
 '4019',
 '40390',
 '412',
 '41401',
 '42731',
 '42789',
 '4280',
 '4589',
 '486',
 '49390',
 '496',
 '51881',
 '53081',
 '5589',
 '5601',
 '56089',
 '5609',
 '56210',
 '56400',
 '56722',
 '5680',
 '5849',
 '5859',
 '5990',
 '71590',
 '73300',
 '73390',
 '78052',
 '78060',
 '78321',
 '7840',
 '7850',
 '78659',
 '78701',
 '78702',
 '78791',
 '78820',
 '78900',
 '79092',
 '99592',
 '99859',
 'D649',
 'E785',
 'E8490',
 'E8497',
 'E8782',
 'E8788',
 'E8798',
 'E9320',
 'F329',
 'F419',
 'I10',
 'K219',
 'K5090',
 'K5190',
 'N179',
 'V1251',
 'V1279',
 'V1582',
 'V442',
 'V4572',
 'V4589',
 'V4986',
 'V552',
 'V5861',
 'V5865',
 'V5866',
 'V5867',
 'V5869',
 'Y929',
 'Z87891',
 'Z9049']

In [16]:
# X
suspect_icd_list = get_suspect_icd_list(100)
# suspect_icd_list = both_ibd_patients
X = np.zeros((len(df_demography.index), 2 + len(suspect_icd_list)))

'''
0: gender | 1: age_group | 2: icd_2724 | 3: icd_27651 | 4: icd_2859 | 5: icd_30000 | 7: icd_311 | 8: icd_4019 | 9: icd_53081 | 10: icd_5849 | 11: icd_V1582
'''


for i in range(len(df_demography.index)):
    subject_id = df_demography.index[i]
    tmp_df = df[df['subject_id'] == subject_id]
    
    X[i, 0] = 1 if df_demography.loc[subject_id, 'gender'] == 'M' else 0
    X[i, 1] = 1 if int(df_demography.loc[subject_id, 'anchor_age']) >= 60 else 0
    # X[i, 2] = 1 if df_demography.loc[subject_id, 'anchor_year_group'] == 'Year 4' else 0
    
    for j in range(len(suspect_icd_list)):
        X[i, 2 + j] = 1 if (tmp_df['icd_code'] == suspect_icd_list[j]).any() else 0

X.shape

(2417, 125)

In [17]:
# y
labelDod = lambda x: 1 if type(x) == str else 0

y = df_demography['dod'].apply(labelDod).values
print('  dead nums: {} live nums: {}'.format(y[y == 1].shape, y[y == 0].shape))

  dead nums: (380,) live nums: (2037,)


In [7]:
# Save files
np.savetxt('../r scripts/data/X.csv', X, delimiter=',', header=','.join(['gender', 'age'] + suspect_icd_list), comments='')
np.savetxt('../r scripts/data/y.csv', y, delimiter=',', header='dod', comments='')

In [8]:
suspect_icd_list

['2724', '27651', '2859', '30000', '311', '4019', '53081', '5849', 'V1582']

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [ ]:
[0, 1, 2] + get_suspect_icd_list(400)

In [ ]:
X = X[:, [1, 3, 5, 7, -2, -1]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print('--> Logistic Regression')
print('  accuracy:\n    {:.4f}'.format(accuracy))
print('  precision:\n    {:.4f}'.format(precision))
print('  recall:\n    {:.4f}'.format(recall))
print('  f1:\n    {:.4f}'.format(f1))


In [ ]:
def logistic_regression(X: np.ndarray, y: np.ndarray, random_state: int = 42, test_size: float = 0.2) -> LogisticRegression:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    model = LogisticRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    print('--> Logistic Regression')
    print('  accuracy:\n    {:.4f}'.format(accuracy))
    print('  precision:\n    {:.4f}'.format(precision))
    print('  recall:\n    {:.4f}'.format(recall))
    print('  f1:\n    {:.4f}'.format(f1))
    
    return model

In [ ]:
model = logistic_regression(X, y)

In [ ]:
model = logistic_regression(X[:, [1, 3, 5, 7, 10, 11]], y, test_size=0.2, random_state=45)

In [ ]:
stats = df.groupby(['subject_id', 'icd_code']).agg({
    'icd_code': ['count'],
})
stats.columns = ['count']
stats = stats.reset_index()
stats = stats.groupby('icd_code').agg({
    'subject_id': ['nunique']
})
stats.columns = ['count']

stats[stats.index == '2761'], stats[stats.index == '2762'], stats[stats.index == '2768']

In [ ]:
get_suspect_icd_list(400)

In [25]:
380 / 2417, 14 / 85, 199 / 1365, 167 / 1137

(0.15721969383533305,
 0.16470588235294117,
 0.1457875457875458,
 0.1468777484608619)